In [ ]:
import pandas as pd
import numpy as np

# produce vector inline graphics
# from IPython.display import set_matplotlib_formats, display, Markdown, HTML

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import NMF

# set_matplotlib_formats('pdf', 'svg')

# import matplotlib.pyplot as plt
# plt.rcParams['figure.figsize'] = [10, 4]
# display(HTML("<style>.container { width:70% !important; }</style>"))

In [ ]:
import os

if not os.path.exists('data'):
    os.mkdir('data')

# Preprocess data for [Voilà](https://github.com/voila-dashboards/voila) dashboard

In the previous example, we turned our exploratory notebook directly into a Voila app. When we started that app, it needed to run every cell in the notebook to produce results. 

This is inefficient. Our end-user shouldn't have to wait for this code to run every time they want to see the dashboard. This becomes especially relevant as we start doing more machine learning, or really any computations that take a considerable time to run. 

In this notebook, we'll reproduce the pre-processing steps needed for the dashboard. We will then save new data and export our ML model.

We need to run this notebook before we run the new, faster Voila app. In a production setting, we could separate these two *services* into separate applications, e.g. two separate Heroku apps connected to the same database.

### Data
We'll be using an [example dataset](https://raw.githubusercontent.com/shubham13p/Ad-Click-Prediction/master/advertising.csv) found on GitHub. We'll use Pandas to load in this data and display the first few rows.

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/shubham13p/Ad-Click-Prediction/master/advertising.csv')
df['Click_labeled'] = df['Clicked on Ad'].apply(lambda x: "Click" if x == 1 else "No Click")
df.head()

## Matplotlib / Altair plots

If we wanted to, we could generate and export the our plots as standalone  files ahead of time. If we took that approach, we could actually embed them in *static* HTML, and we might not even need Voila. This is even true for the interactive plots!

We won't go into too much detail about this, but Altair has the benefit of creating charts that run completely on Javascript. As such, they don't need any real backend to run. We could have rich, interactive charts even on something like GitHub Pages, or any static HTML server.

So, again, **why use Voila?**

**Pros:**
* Integrates directly with Jupyter workflow
* Less overhead - we don't need to build a separate app or any other infrastructure

**Cons:**
* Slower to run
* Less flexibility than other frameworks

So the main use-case for Voila is in **protyping** a report, app, or dashboard. 

## Topic Modeling

Here, we'll handle the ML part of the app and add those features to the DataFrame. If we wanted to, we could save the topic model and load it in our Voila app, but we don't really need to. 

**Q: When would we want to include the model in the Voila app?** A: If we needed to generate predictions interactively. We're just using the model for descriptive analysis, rather than predictive, so we just need to save the results. 

In [ ]:
tfidf_vect = TfidfVectorizer(max_df=0.9, min_df=10, stop_words='english')

# Create matrix of TFIDF features
doc_term_matrix = tfidf_vect.fit_transform(df['Ad Topic Line'].values.astype('U'))

In [ ]:
# Create NMF model
nmf = NMF(n_components=3, random_state=42)
nmf.fit(doc_term_matrix )

# Create matrix of NMF outputs
topic_values = nmf.transform(doc_term_matrix)

# Assign the most relevant topic to each row in the DataFrame
df['Topic'] = topic_values.argmax(axis=1) + 1

df.head()

In [ ]:
df.to_csv('data/df.csv')

In [ ]:
nmf_components = nmf.components_

np.save('data/nmf-components.npy', nmf_components)

In [ ]:
feature_names = tfidf_vect.get_feature_names()

with open('data/feature-names.txt', 'w') as f:
    f.write('\n'.join(feature_names))
